<a href="https://colab.research.google.com/github/HisakaKoji/bert-japanese/blob/master/finetune_to_livedoor_corpus_20200103titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetuning of the pretrained Japanese BERT model

Finetune the pretrained model to solve multi-class classification problems.  
This notebook requires the following objects:
- trained sentencepiece model (model and vocab files)
- pretraiend Japanese BERT model

Dataset is livedoor ニュースコーパス in https://www.rondhuit.com/download.html.  
We make test:dev:train = 2:2:6 datasets.

**This colab notebook assumes the above models are stored on some GSC bucket you can acess its objects.**

In [0]:
import pandas as pd
df = pd.read_csv('train.csv', sep=',')
train['text'] = df['PassengerId'].astype(str) + ':' +  df['Pclass'].astype(str) + ':' +  df['Name'].astype(str) + ':' + df['Sex'].astype(str)  + ':' + df['Age'].astype(str) + ':' + df['SibSp'].astype(str)
train['label'] = df['Survived']
train.to_csv('bert_titanic_20200103.csv', index=None)

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df['text'],df['label'],test_size = 0.2 )
train = pd.DataFrame()
train['label'] = y_train
train['text'] = X_train
train.to_csv('train.csv',index=None)
valid = pd.DataFrame()
valid['label'] = y_test
valid['text'] = X_test
valid.to_csv('dev.csv',index=None)





In [92]:
!git clone --recursive https://github.com/HisakaKoji/bert-japanese.git

fatal: destination path 'bert-japanese' already exists and is not an empty directory.


In [0]:
!pip install -q -r bert-japanese/requirements.txt

In [0]:
from google.colab import auth
auth.authenticate_user()

In [138]:
%cd ..

/content/bert-japanese


In [139]:
!git pull

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/HisakaKoji/bert-japanese
   36a5ac3..bec991c  master     -> origin/master
Updating 36a5ac3..bec991c
Fast-forward
 src/run_classifier.py | 6 +++++-
 1 file changed, 5 insertions(+), 1 deletion(-)


In [140]:
%cd notebook

/content/bert-japanese/notebook


In [57]:
!gsutil cp gs://hisaka/model/wiki-ja.model ../model/
!gsutil cp gs://hisaka/model/wiki-ja.vocab ../model/

Copying gs://hisaka/model/wiki-ja.model...
- [1 files][786.8 KiB/786.8 KiB]                                                
Operation completed over 1 objects/786.8 KiB.                                    
Copying gs://hisaka/model/wiki-ja.vocab...
/ [1 files][581.7 KiB/581.7 KiB]                                                
Operation completed over 1 objects/581.7 KiB.                                    


In [51]:
%ls

check-extract-features.ipynb       finetune-to-livedoor-corpus.ipynb
check-trained-tokenizer.ipynb      pretraining.ipynb
finetune_to_livedoor_corpus.ipynb


Check TPU.

In [91]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is grpc://10.64.241.74:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 11537326700386695193),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 7892721330015887097),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 14109537377276890775),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 14348902840641519132),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 5506530810611582237),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 6790657936099474827),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 7081938252929965427),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 12953066637652651743),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 802727606604

## Data preparation

You need to put preprocessed data on your GCS bucket.  
To create preprocessed data, follow https://github.com/yoheikikuta/bert-japanese/blob/master/notebook/finetune-to-livedoor-corpus.ipynb.

## Finetune pre-trained model

In [0]:

PRETRAINED_MODEL_PATH = 'gs://hisaka/rurubu/output/model.ckpt-419'  # GCS bucket
PRETRAINED_MODEL_PATH = 'gs://hisaka/model/model.ckpt-1400000'  # GCS bucket
INPUT_DATA_GCS = 'gs://hisaka/20200103titanicBert'  # GCS bucket
FINETUNE_OUTPUT_DIR = 'gs://hisaka/20190103titanicOut' # GCS bucket

In [89]:
!ls ../model

wiki-ja.model  wiki-ja.vocab


In [141]:
%%time
!python3 ../src/run_classifier.py \
  --task_name=titanic \
  --use_tpu=True \
  --tpu_name={TPU_ADDRESS} \
  --do_train=true \
  --do_eval=true \
  --data_dir={INPUT_DATA_GCS} \
  --model_file=../model/wiki-ja.model \
  --vocab_file=../model/wiki-ja.vocab \
  --init_checkpoint={PRETRAINED_MODEL_PATH} \
  --max_seq_length=512 \
  --train_batch_size=64 \
  --learning_rate=2e-5 \
  --num_train_epochs=10.0 \
  --output_dir={FINETUNE_OUTPUT_DIR}




W0103 07:21:15.227315 139985066841984 module_wrapper.py:139] From ../src/run_classifier.py:701: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0103 07:21:15.227553 139985066841984 module_wrapper.py:139] From ../src/run_classifier.py:701: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0103 07:21:15.228000 139985066841984 module_wrapper.py:139] From /content/bert-japanese/src/../bert/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0103 07:21:15.228913 139985066841984 module_wrapper.py:139] From ../src/run_classifier.py:723: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

Loaded a trained SentencePiece model.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * h

## Predict using the finetuned model

Let's predict test data using the finetuned model.  

In [142]:
%%time
!python3 ../src/run_classifier.py \
  --task_name=titanic \
  --use_tpu=True \
  --tpu_name={TPU_ADDRESS} \
  --do_train=false \
  --do_eval=false \
  --do_predict=true \
  --data_dir={INPUT_DATA_GCS} \
  --model_file=../model/wiki-ja.model \
  --vocab_file=../model/wiki-ja.vocab \
  --init_checkpoint={PRETRAINED_MODEL_PATH} \
  --max_seq_length=512 \
  --train_batch_size=64 \
  --learning_rate=2e-5 \
  --num_train_epochs=3.0 \
  --output_dir={FINETUNE_OUTPUT_DIR}




W0103 07:22:14.209877 140239731148672 module_wrapper.py:139] From ../src/run_classifier.py:701: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0103 07:22:14.210087 140239731148672 module_wrapper.py:139] From ../src/run_classifier.py:701: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0103 07:22:14.210475 140239731148672 module_wrapper.py:139] From /content/bert-japanese/src/../bert/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0103 07:22:14.211270 140239731148672 module_wrapper.py:139] From ../src/run_classifier.py:723: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

Loaded a trained SentencePiece model.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * h

## Evaluation

Download result and original data.

In [143]:
!gsutil cp {FINETUNE_OUTPUT_DIR}/test_results.tsv .
!gsutil cp {INPUT_DATA_GCS}/train.tsv .
!gsutil cp {INPUT_DATA_GCS}/dev.tsv .
!gsutil cp {INPUT_DATA_GCS}/test.tsv .

Copying gs://hisaka/20190103titanicOut/test_results.tsv...
/ [1 files][  9.4 KiB/  9.4 KiB]                                                
Operation completed over 1 objects/9.4 KiB.                                      
Copying gs://hisaka/20200103titanicBert/train.tsv...
/ [1 files][ 33.8 KiB/ 33.8 KiB]                                                
Operation completed over 1 objects/33.8 KiB.                                     
Copying gs://hisaka/20200103titanicBert/dev.tsv...
/ [1 files][  8.5 KiB/  8.5 KiB]                                                
Operation completed over 1 objects/8.5 KiB.                                      
Copying gs://hisaka/20200103titanicBert/test.tsv...
/ [1 files][ 20.4 KiB/ 20.4 KiB]                                                
Operation completed over 1 objects/20.4 KiB.                                     


### Trained model

Check accuracy.

In [0]:
import numpy as np
import pandas as pd

In [150]:
import sys
sys.path.append("../src")

from run_classifier import livedoorProcessor

processor = livedoorProcessor()
label_list = processor.get_labels()

ImportError: ignored

In [104]:
label_list

['Traditional-Festivalsand-annual-events',
 'Shrine-floats-etc.',
 'Traditional-performing-arts-and-dance',
 'Procession-and-parade',
 'food',
 'market',
 'flower-nature',
 'fire',
 'fireworks',
 'snow',
 'illumination',
 'music',
 'sports',
 'museum',
 'museum',
 'festival',
 'animal',
 'experience',
 'school',
 'talk',
 'stage',
 'thema-park',
 'animal-fish-park',
 'anniversary',
 'fair',
 'other',
 'Industry',
 'Customs',
 'Agricultural ritual',
 'none']

In [0]:
label_list = ['0','1']

In [0]:
result = pd.read_csv("./test_results.tsv", sep='\t', header=None)

In [153]:
result.head(2)

,0,1
0,0.998936,0.001064
1,0.992690,0.007310


In [0]:
test_df = pd.read_csv("./test.tsv", sep='\t')

In [155]:
label_list

['0', '1']

In [0]:
test_df['predict'] = [ label_list[np.array(elem[1]).argmax()] for elem in result.iterrows() ]

In [157]:
sum( test_df['label'] == test_df['predict'] ) / len(test_df)

0.0

In [0]:
### 1/5 of full training data.
# sum( test_df['label'] == test_df['predict'] ) / len(test_df)

In [0]:
test_df.to_csv('titanic_bert_result2.csv')

In [159]:
test_df

,text,label,predict
0,"892:3:Kelly, Mr. James:male:34.5:0",0,0
1,"893:3:Wilkes, Mrs. James (Ellen Needs):female:...",0,0
2,"894:2:Myles, Mr. Thomas Francis:male:62.0:0",0,0
3,"895:3:Wirz, Mr. Albert:male:27.0:0",0,0
4,"896:3:Hirvonen, Mrs. Alexander (Helga E Lindqv...",0,1
...,...,...,...
413,"1305:3:Spector, Mr. Woolf:male:nan:0",0,0
414,"1306:1:Oliva y Ocana, Dona. Fermina:female:39.0:0",0,1
415,"1307:3:Saether, Mr. Simon Sivertsen:male:38.5:0",0,0
416,"1308:3:Ware, Mr. Frederick:male:nan:0",0,0


A littel more detailed check using `sklearn.metrics`.

In [0]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [122]:
print(classification_report(test_df['label'].astype('str'), test_df['predict'].astype('str')))

              precision    recall  f1-score   support

           0       1.00      0.64      0.78       418
           1       0.00      0.00      0.00         0

    accuracy                           0.64       418
   macro avg       0.50      0.32      0.39       418
weighted avg       1.00      0.64      0.78       418



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [0]:
### 1/5 of full training data.
# print(classification_report(test_df['label'], test_df['predict']))

In [125]:
print(confusion_matrix(test_df['label'].astype('str'), test_df['predict'].astype('str'),))

[[267 151]
 [  0   0]]


In [0]:
### 1/5 of full training data.
# print(confusion_matrix(test_df['label'], test_df['predict']))

In [0]:
!ls

check-extract-features.ipynb	   pretraining.ipynb
check-trained-tokenizer.ipynb	   test_results.tsv
dev.tsv				   test.tsv
finetune_to_livedoor_corpus.ipynb  train.tsv
finetune-to-livedoor-corpus.ipynb


In [0]:
test_df.to_csv('../test20191221.csv')

In [0]:
!gsutil cp -r  ../test20191221.csv  gs://hisaka/20191220 

Copying file://../test20191221.csv [Content-Type=text/csv]...
-
Operation completed over 1 objects/423.1 KiB.                                    


### Simple baseline model.

In [0]:
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [0]:
train_df = pd.read_csv("./train.tsv", sep='\t')
dev_df = pd.read_csv("./dev.tsv", sep='\t')
test_df = pd.read_csv("./test.tsv", sep='\t')

In [128]:
!apt-get -q install -y mecab libmecab-dev mecab-ipadic mecab-ipadic-utf8

Reading package lists...
Building dependency tree...
Reading state information...
libmecab-dev is already the newest version (0.996-5).
mecab is already the newest version (0.996-5).
mecab-ipadic is already the newest version (2.7.0-20070801+main-1).
mecab-ipadic-utf8 is already the newest version (2.7.0-20070801+main-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.


In [0]:
!pip install -q mecab-python3==0.7

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import GradientBoostingClassifier
import MeCab

In [0]:
m = MeCab.Tagger("-Owakati")

In [0]:
train_dev_df = pd.concat([train_df, dev_df])

In [0]:
train_dev_xs = train_dev_df['text'].apply(lambda x: m.parse(x))
train_dev_ys = train_dev_df['label']

test_xs = test_df['text'].apply(lambda x: m.parse(x))
test_ys = test_df['label']

In [0]:
vectorizer = TfidfVectorizer(max_features=750)
train_dev_xs_ = vectorizer.fit_transform(train_dev_xs)
test_xs_ = vectorizer.transform(test_xs)

The following set up is not exactly identical to that of BERT because inside Classifier it uses `train_test_split` with shuffle.  
In addition, parameters are not well tuned, however, we think it's enough to check the power of BERT.

In [135]:
%%time

model = GradientBoostingClassifier(n_estimators=200,
                                   validation_fraction=len(dev_df)/len(train_df),
                                   n_iter_no_change=5,
                                   tol=0.01,
                                   random_state=23)

### 1/5 of full training data.
# model = GradientBoostingClassifier(n_estimators=200,
#                                    validation_fraction=len(train_df)/len(dev_df),
#                                    n_iter_no_change=5,
#                                    tol=0.01,
#                                    random_state=23)

model.fit(train_dev_xs_, train_dev_ys)

CPU times: user 58.9 ms, sys: 4.15 ms, total: 63 ms
Wall time: 76.8 ms


In [136]:
print(classification_report(test_ys, model.predict(test_xs_)))

              precision    recall  f1-score   support

           0       1.00      0.59      0.74       418
           1       0.00      0.00      0.00         0

    accuracy                           0.59       418
   macro avg       0.50      0.29      0.37       418
weighted avg       1.00      0.59      0.74       418



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [0]:
### 1/5 of full training data.
# print(classification_report(test_ys, model.predict(test_xs_)))

In [137]:
print(confusion_matrix(test_ys, model.predict(test_xs_)))

[[245 173]
 [  0   0]]


In [0]:
### 1/5 of full training data.
# print(confusion_matrix(test_ys, model.predict(test_xs_)))